In [ ]:
%matplotlib inline
from matplotlib import pyplot
import geopandas
from shapely.geometry import Polygon

%run -i weave_map.py

In [ ]:
ak = geopandas.read_file("../data/imd-auckland-2018.gpkg")

In [ ]:
import copy

max_coord = 150

x1 = [x for x in range(-max_coord, max_coord + 1, 1)]
y1 = [np.cos(i / max_coord * np.pi) * 50 for i in x1]
x2 = copy.copy(x1)
y2 = copy.copy(y1)

x1.reverse()
y1.reverse()
x1.insert(0, max_coord)
x1.insert(0, -max_coord)
y1.insert(0, max_coord)
y1.insert(0, max_coord)

x2.insert(0, -max_coord)
x2.insert(0, max_coord)
y2.insert(0, -max_coord)
y2.insert(0, -max_coord)

p1 = Polygon(zip(x1, y1))
p2 = Polygon(zip(x2, y2))

In [ ]:
tile1 = geopandas.GeoSeries([p1, p2])
tile2 = tile1.geometry.rotate(90, (0, 0))
tile1 = geopandas.GeoDataFrame(geometry = tile1, crs = 2193)
tile2 = geopandas.GeoDataFrame(geometry = tile2, crs = 2193)

In [ ]:
tiles = tile2.overlay(tile1)
tiles["strand"] = list("abcd")

In [ ]:
tile_bb = geopandas.GeoDataFrame(
    geometry = geopandas.GeoSeries([tiles.unary_union.envelope]), crs = 2193)
tile_bb.plot()                                

In [ ]:
tiles.geometry = tiles.geometry.buffer(-10)
tiles.plot(column = "strand")

In [ ]:
unit = WeaveUnit(crs = 2193) # just to get the container
unit.elements = tiles
unit.tile = tile_bb

In [ ]:
tiling = Tiling(unit, ak)
t30 = ak.overlay(tiling.rotated(30))

In [ ]:
fig = pyplot.figure(figsize = (24, 15))
ax = fig.add_subplot(111)
ak.plot(ax = ax, facecolor = "grey", linewidth = 0)

strands = set(t30.strand)
cols = [x for x in t30.columns if "Rank" in x]
cmaps = ("viridis", "plasma_r", "Spectral", "PRGn")

for strand, col, cmap in zip(strands, cols, cmaps):
    t30[t30.strand == strand].plot(ax = ax, column = col, cmap = cmap, k = 100, scheme = "equal_interval", linewidth = 0)


In [ ]:
t30.to_file("test.gpkg")